In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [44]:
df = pd.read_csv('clientes-v2.csv')
df.head()

,nome,cpf,idade,data,endereco,estado,pais,salario,nivel_educacao,numero_filhos,estado_civil,anos_experiencia,area_atuacao
0,Vitor Lima,612.570.493-99,56,19/01/1968,"Estrada Barbosa, 255\nVila Tirol\n53133-647 Ba...",Pará,Brasil,13550.54,Ensino Médio,0,Casado,17,Tecnologia
1,Ana Clara Martins,648.129.530-06,33,02/12/1990,"Favela Duarte, 44\nBonfim\n37709685 Farias das...",Santa Catarina,Brasil,10645.16,NaN,3,NaN,15,NaN
2,Heloísa da Mata,340.981.256-33,49,31/01/1975,"Pátio de Vieira, 83\nJonas Veiga\n53832-319 Ca...",Rio Grande do Norte,Brasil,3124.72,Ensino Médio,3,Casado,3,Indústria
3,Juan Novaes,097.168.524-01,54,26/08/1969,Morro de Pereira\nPiraja\n36797-897 Rocha / PA,Minas Gerais,Brasil,7534.60,Ensino Médio,0,Casado,18,Tecnologia
4,Srta. Sarah Peixoto,079.283.516-68,61,19/03/1963,"Área Marina Martins, 34\nSão Sebastião\n10696-...",Amapá,Brasil,4067.73,Ensino Fundamental,1,Solteiro,16,Comércio


In [45]:
# dropando a coluna pais pois ela não tem valor para a análise
df = df.drop('pais', axis=1)

#ajustando as datas 
df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y',errors='coerce')

In [46]:
# Verificando e tratando os nulos
df.isnull().sum()

nome                  0
cpf                   0
idade                 0
data                  0
endereco              0
estado              585
salario             572
nivel_educacao      454
numero_filhos         0
estado_civil        454
anos_experiencia      0
area_atuacao        454
dtype: int64

In [47]:
# usando endereço para arrumar os estados

dic_estados = {"SP": "São Paulo", "RJ": "Rio de Janeiro", "MG": "Minas Gerais", "ES": "Espírito Santo",
                "AM": "Amazonas", "BA": "Bahia", "CE": "Ceará", "DF": "Distrito Federal", "GO": "Goiás",
                  "MA": "Maranhão", "MT": "Mato Grosso", "MS": "Mato Grosso do Sul", "PA": "Pará",
                    "PB": "Paraíba", "PR": "Paraná", "PE": "Pernambuco", "PI": "Piauí", "RN": "Rio Grande do Norte",
                      "RS": "Rio Grande do Sul", "RO": "Rondônia", "RR": "Roraima", "SC": "Santa Catarina",
                        "SE": "Sergipe", "TO": "Tocantins", "AC": "Acre", "AL": "Alagoas", "AP": "Amapá"}

def preencher_nulos_estados(df):
    mask = df['estado'].isnull()  # Máscara para linhas onde estado é NaN

    df.loc[mask, 'estado'] = df.loc[mask, 'endereco'].apply(
        lambda x: x.split('/')[1].strip() if '/' in x else None
    )

    mask = df['estado'].apply(lambda x: x in dic_estados.keys())
    df.loc[mask, 'estado'] = df.loc[mask, 'estado'].apply(lambda x: dic_estados[x])
    return df


df = preencher_nulos_estados(df)

In [48]:
# ajustando os nulos
df['salario'] = df['salario'].fillna(df['salario'].mean())

df['nivel_educacao'] = df['nivel_educacao'].fillna('Não informado')

df['estado_civil'] = df['estado_civil'].fillna('Não informado') 

df['area_atuacao'] = df['area_atuacao'].fillna('Não informado')

# checando novamente o numero de nulos
df.isnull().sum()

nome                0
cpf                 0
idade               0
data                0
endereco            0
estado              0
salario             0
nivel_educacao      0
numero_filhos       0
estado_civil        0
anos_experiencia    0
area_atuacao        0
dtype: int64

In [ ]:
# arrumando o endereco e criando as colunas rua, bairro, cidade e cep

def separando_endereco(address):
    parts = address.split('\n')
    return (parts[0] if len(parts) > 0 else None,
            parts[1] if len(parts) > 1 else None,
            parts[2] if len(parts) > 2 else None)

df['rua'], df['bairro'], df['cep'] = zip(*df['endereco'].apply(separando_endereco))

# criando a coluna cidade
df['cidade'] = df['cep'].apply(lambda x: x.split('/')[0].split(' ', 1)[1] if ' ' in x else 'Não informado')

# ajustando e formatando o cep
df['cep'] = df['cep'].apply(lambda x: x.split(' ')[0])

def formatar_cep(cep):
    cep = str(cep)  
    if len(cep) == 8:  
        return f"{cep[:5]}-{cep[5:]}"  
    return cep  

df['cep'] = df['cep'].apply(formatar_cep)

# dropando a coluna endereco
df = df.drop('endereco', axis=1)

df.head()

,nome,cpf,idade,data,estado,salario,nivel_educacao,numero_filhos,estado_civil,anos_experiencia,area_atuacao,rua,bairro,cep,cidade
0,Vitor Lima,612.570.493-99,56,1968-01-19,Pará,13550.54,Ensino Médio,0,Casado,17,Tecnologia,"Estrada Barbosa, 255",Vila Tirol,53133-647,Barros
1,Ana Clara Martins,648.129.530-06,33,1990-12-02,Santa Catarina,10645.16,Não informado,3,Não informado,15,Não informado,"Favela Duarte, 44",Bonfim,37709-685,Farias das Flores
2,Heloísa da Mata,340.981.256-33,49,1975-01-31,Rio Grande do Norte,3124.72,Ensino Médio,3,Casado,3,Indústria,"Pátio de Vieira, 83",Jonas Veiga,53832-319,Cardoso
3,Juan Novaes,097.168.524-01,54,1969-08-26,Minas Gerais,7534.60,Ensino Médio,0,Casado,18,Tecnologia,Morro de Pereira,Piraja,36797-897,Rocha
4,Srta. Sarah Peixoto,079.283.516-68,61,1963-03-19,Amapá,4067.73,Ensino Fundamental,1,Solteiro,16,Comércio,"Área Marina Martins, 34",São Sebastião,10696-521,das Neves


In [54]:
# criando uma mascara para os cpfs

df['cpf'] = df['cpf'].apply(lambda x: x[:3] +'.***.***-'+ x[-2:])    

# salvando o novo dataset
df.to_csv('clientes_tratados.csv', index=False)